# Import Packages

In [1]:
options(warn = -1) # options(warn = 0)

In [2]:
check.packages = function(pkg) {
    new.pkg = pkg[!(pkg %in% installed.packages()[, "Package"])]
    if (length(new.pkg)) 
        install.packages(new.pkg, dependencies = TRUE)
    sapply(pkg, require, character.only = TRUE)
}

In [3]:
check.packages(
    c(
        ## Procurement & Querying
        'RPostgreSQL',
        'httr',
        'jsonlite',
        
        ## Storing
        'googlesheets',
        
        ## Cleaning & Processing
        'dplyr',
        'lubridate',
        'stringr',
        
        ## Visualizing
        'ggplot2'
    )
)

Loading required package: RPostgreSQL
Loading required package: DBI
Loading required package: httr
Loading required package: jsonlite
Loading required package: googlesheets
Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: stringr
Loading required package: ggplot2


RPostgreSQL         httr     jsonlite googlesheets        dplyr    lubridate 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
     stringr      ggplot2 
        TRUE         TRUE

# Import Data

In [4]:
# ## WORLD TRADING DATA


# googl.get = GET(
#     url = paste(
#         'https://www.worldtradingdata.com/api/v1/history?symbol=GOOGL&api_token=',
#         Sys.getenv('WORLD_TRADING_DATA_API_KEY'),
#         '&date_to=2019-04-14&sort=oldest&output=json'
#     ), 
#     sep = ''
# )
# # headers(googl.get)
# googl.get$status_code
# # str(content(googl.get, 'parsed'))

In [5]:
# googl.hist = content(googl.get, 'parsed')$history

# ## Convert googl.hist to be a data frame of size R x C = Dates x Attributes

# googl = data.frame(
#     session = names(googl.hist),
#     matrix(
#         data = unlist(googl.hist),
#         nrow = length(googl.hist),
#         byrow = TRUE
#     ),
#     stringsAsFactors = FALSE
# )



# ## Since all dates contain the same number of attributes, can just pick random date to get attribute names.

# colnames(googl) = c(
#     'session',
#     names(googl.hist$'2004-08-19')
# )



# ## Convert Data Types

# googl$session = as.Date(googl$session)
# googl$open = as.numeric(googl$open)
# googl$close = as.numeric(googl$close)
# googl$high = as.numeric(googl$high)
# googl$low = as.numeric(googl$low)
# googl$volume = as.numeric(googl$volume)



# str(googl)
# head(googl)

# Upsert to Google Sheets

In [8]:
a = getStockData('BYND')

[1] 200


In [10]:
a %>% str()

'data.frame':	75 obs. of  6 variables:
 $ session: Date, format: "2019-08-16" "2019-08-15" ...
 $ open   : num  144 163 166 169 165 ...
 $ close  : num  145 144 163 167 169 ...
 $ high   : num  149 164 167 172 172 ...
 $ low    : num  136 137 161 166 162 ...
 $ volume : num  5920368 9638551 2246577 3053719 3973805 ...


In [6]:
getStockDailyHTTP = function (ticker, timestart = NA, timeend = NA) {
    
    paste(
        'https://www.worldtradingdata.com/api/v1/history?symbol=',
        ticker,
        '&api_token=',
        Sys.getenv('WORLD_TRADING_DATA_API_KEY'),
        ifelse(
            !is.na(timestart), 
            paste('&date_from=', timestart, sep = ''),
            ''
        ),
        ifelse(
            !is.na(timeend), 
            paste('&date_end=', timeend, sep = '')
            ,''
        ),
        sep = ''
    )
}

In [7]:
getStockData = function (ticker, timestart = NA, timeend = NA) {
    
    
    ## Set API call parameters.
#     timestart = ifelse(
#         ticker %in% (getGoogleSheet() %>% getGoogleSheetTabs()), #If new stock, cannot have a sheet with that ticker name in the spreadsheet.
#         getGoogleSheet() %>% getLatestDate(ticker = ticker),
#         timestart
#     )
#     timeend = min(c(timeend, Sys.Date()))
        
    
    
    ## Make API Call.
    ticker.get = GET(
        url = getStockDailyHTTP(
            ticker = ticker,
            timestart = timestart,
            timeend = timeend
        )
    )
    print(ticker.get$status_code)
    
    
    
    ## Get JSON Output to Data Frame.
    ticker.hist = content(ticker.get, 'parsed')$history
    ticker.data = data.frame(
        session = names(ticker.hist),
        matrix(
            data = unlist(ticker.hist),
            nrow = length(ticker.hist),
            byrow = TRUE
        ),
        stringsAsFactors = FALSE
    )
    
    
    
    ## Convert column names and data types.
    colnames(ticker.data) = c(
        'session',
        names(ticker.hist[[1]])
    )


    ticker.data$session = as.Date(ticker.data$session)
    ticker.data$open = as.numeric(ticker.data$open)
    ticker.data$close = as.numeric(ticker.data$close)
    ticker.data$high = as.numeric(ticker.data$high)
    ticker.data$low = as.numeric(ticker.data$low)
    ticker.data$volume = as.numeric(ticker.data$volume)
    
    return(ticker.data)
    
}

In [9]:
getGoogleSheet = function (gs.name = 'stock-database') {
    
    stocks.gs = gs_key(
        gs_title(gs.name)$sheet_key
    )
    
    # gs_auth()
    # gs_user()
    
    return(stocks.gs)
    
}

In [10]:
getGoogleSheetRefresh = function (gs.object) {
    
    gs.object %>% gs_gs()    
}

In [11]:
getGoogleSheetTabs = function (gs.object) {
    
    gs.object %>% gs_ws_ls()
}

In [12]:
# createGoogleSheetTab = function (gs.object, ticker, ticker.data) {
createGoogleSheetTab = function (gs.object, ticker) {
    
    gs.object %>% 
        gs_ws_new(
            ws_title = ticker,
#             input = ticker.data,
            verbose = TRUE
        )
}

In [13]:
deleteGoogleSheetTab = function (gs.object, ticker) {
    
    ws.ls = gs.object %>% getGoogleSheetTabs()
    
    gs.object %>% 
        gs_ws_delete(
            ws = which(ws.ls == ticker)
        )
}

In [14]:
insertStockPrices = function (gs.object, ticker, ticker.data) {
    
    ws.ls = gs.object %>% getGoogleSheetTabs()
    
    current.ws.data = gs.object %>% gs_read(ws = ticker)
    
    if (nrow(current.ws.data) == 0) {
    
        gs.object %>%
            gs_add_row(
                ws = which(ws.ls == ticker),
                input = head(ticker.data),
                verbose = TRUE
            )
    
    } else {
        
        gs.object %>%
            gs_add_row(
                ws = which(ws.ls == ticker),
                input = anti_join( #historical data is immutable - don't have to worry about updates.
                    ticker.data,
                    current.ws.data
                ),
                verbose = TRUE
            )

    }
    
}

In [15]:
getLatestDate = function (gs.object, ticker) {
    
    max(as.Date((gs.object %>% gs_read(ws = ticker))$session))
}

In [16]:
getEarliestDate = function (gs.object, ticker) {
    
    min(as.Date((gs.object %>% gs_read(ws = ticker))$session))
}

In [25]:
etlToGoogleSheets = function (ticker, timestart = NA, timeend = NA) {
    
    gs.object = getGoogleSheet()
    
    ticker.data = getStockData(
        ticker = ticker,
        timestart = timestart,
        timeend = timeend
    ) %>%
        arrange(
            session
        )
    
    ws.ls = gs.object %>% getGoogleSheetRefresh() %>% getGoogleSheetTabs()
    
    if (ticker %in% (ws.ls)) {
        
       gs.object %>% insertStockPrices(
           ticker = ticker,
           ticker.data = ticker.data
       ) 
        
    } else {
        
        a = gs.object %>% 
            createGoogleSheetTab(
                ticker = ticker
            ) 

        ws.ls = gs.object %>% getGoogleSheetRefresh() %>% getGoogleSheetTabs()

        a %>%
            gs_edit_cells(
                ws = which(ws.ls == ticker),
                input = head(ticker.data,1),
                anchor = 'A1',
                byrow = TRUE
            ) %>%
            insertStockPrices(
                ticker = ticker,
                ticker.data = ticker.data
            )
        
    }
}

In [ ]:
start = Sys.time()

etlToGoogleSheets('BA')

Sys.time() - start

Auto-refreshing stale OAuth token.
Sheet successfully identified: "stock-database"
Sheet successfully identified: "stock-database"


[1] 200


Sheet successfully identified: "stock-database"
Accessing worksheet titled 'BA'.
Parsed with column specification:
cols(
  session = col_date(format = ""),
  open = col_double(),
  close = col_double(),
  high = col_double(),
  low = col_double(),
  volume = col_double()
)
Joining, by = c("session", "open", "close", "high", "low", "volume")
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row succe

Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
Row successfully appended.
R

In [ ]:
start = Sys.time()

etlToGoogleSheets('MSFT')

Sys.time() - start

In [ ]:
start = Sys.time()

etlToGoogleSheets('WMT')

Sys.time() - start